## ETL: Cohort Construction

### ATTENTION

#### The 1st cell need to have connection to mimic postgres database to run. Put your own connection information in "psycopg2.connect" part

In [ ]:
import psycopg2
import pandas as pd
import os

# connect to database
conn = psycopg2.connect("dbname='mimic' user='postgres' host='40.76.205.12' password='Jbz19860501~'")

# get necessary tables

# patients with all antibiotic prescriptions
prescr_ab_only_query = r"""  select pr.hadm_id as HADM_ID
  , pr.drug as antibiotic_name
  , pr.startdate as antibiotic_time
  , pr.enddate as antibiotic_endtime
  from mimiciii.prescriptions pr
  -- inner join to subselect to only antibiotic prescriptions
  inner join mimiciii.abx_poe_list ab
      on pr.drug = ab.drug"""
prescr_ab_only_df = pd.read_sql_query(prescr_ab_only_query, conn)

# microbiologyevents initial
micro_events_query = r"""select hadm_id as HADM_ID
    , chartdate, charttime
    , spec_type_desc
    , max(case when org_name is not null and org_name != '' then 1 else 0 end) as PositiveCulture
  from mimiciii.microbiologyevents
  group by hadm_id, chartdate, charttime, spec_type_desc"""
micro_events_df = pd.read_sql_query(micro_events_query, conn)


conn.close()

In [2]:
path_folder = r"/mnt/host/home/jbzhang/mimic"

# Patients
patients_df = pd.read_csv(os.path.join(path_folder, 'PATIENTS.csv'))
patients_df['DOD_COALESCE'] = patients_df[['DOD', 'DOD_HOSP', 'DOD_SSN']].bfill(axis=1).iloc[:,0]
patients_df = patients_df[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD_COALESCE', 'EXPIRE_FLAG']]

# ICUSTAYS
icustays_df = pd.read_csv(os.path.join(path_folder, 'ICUSTAYS.csv')).drop(['ROW_ID'], axis=1)
icustays_df = icustays_df[(icustays_df['INTIME'].notnull()) & (icustays_df['OUTTIME'].notnull())] # exclude bad data

# Admissions Info
admissions_df = pd.read_csv(os.path.join(path_folder, 'ADMISSIONS.csv'))[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 
                                                                          'DISCHTIME', 'DEATHTIME', 'DIAGNOSIS', 
                                                                          'HAS_CHARTEVENTS_DATA']]
admissions_df = admissions_df[admissions_df['HAS_CHARTEVENTS_DATA']==1] # exclude no chartevents situation

# Services Info, determining the type of service in hospital
services_df = pd.read_csv(os.path.join(path_folder, 'SERVICES.csv'))[['SUBJECT_ID', 'HADM_ID', 'CURR_SERVICE']]

In [3]:
cohort_df = pd.merge(pd.merge(pd.merge(admissions_df, patients_df, how='inner', on='SUBJECT_ID'), 
                             icustays_df, how='inner', on=['HADM_ID','SUBJECT_ID']), 
                     services_df, how='inner', on=['HADM_ID','SUBJECT_ID'])

In [4]:
cohort_df[['ICUSTAY_ID']].drop_duplicates().shape

(61021, 1)

STEP 1 Add indicator for data source. Using only metavision data

In [5]:
icustays_df['indicator_dbsource'] = icustays_df['DBSOURCE'].apply(lambda x: 1 if x=='metavision' else 0)
beforeRows = len(icustays_df['indicator_dbsource'])
icustays_df = icustays_df.loc[icustays_df['indicator_dbsource'] == 1]
afterRows = len(icustays_df['indicator_dbsource'])
print("Just removed " + str(beforeRows - afterRows) + " Carevue ICU stays.")

Just removed 37902 Carevue ICU stays.


Step 2: Add indicator for adults, using criterion age between 16 and 100

In [6]:

date_format = '%Y-%m-%d %H:%M:%S'
patient_age_df = patients_df.merge(admissions_df, how = "inner", on = "SUBJECT_ID")
print(patient_age_df.columns.values)
patient_age_df["age"] = (pd.to_datetime(patient_age_df['ADMITTIME'], format=date_format) 
                                  - pd.to_datetime(patient_age_df['DOB'], format=date_format)).astype('timedelta64[Y]')
patient_age_df['age'] = (abs(pd.to_datetime(patient_age_df['ADMITTIME'], format=date_format) 
                                  - pd.to_datetime(patient_age_df['DOB'], format=date_format))).astype('timedelta64[Y]')
beforePatients = patient_age_df["SUBJECT_ID"].nunique()

patient_age_df = patient_age_df[patient_age_df["age"] >= 16]
patient_age_df = patient_age_df[patient_age_df["age"] <= 100]
afterPatients = patient_age_df["SUBJECT_ID"].nunique()
print(patients_df.columns.values)
print(patient_age_df.columns.values)
patients_df = patient_age_df[["SUBJECT_ID", "GENDER", "EXPIRE_FLAG", "HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME", "DIAGNOSIS", "age"]]
patients_df = patients_df.drop_duplicates()
print(max(patients_df["age"]))
print(min(patients_df["age"]))

print("Just removed " + str(beforePatients - afterPatients) + " patients who were older than 89 or younger than 16, " + str(afterPatients) + " patients left in the dataset.")

['SUBJECT_ID' 'GENDER' 'DOB' 'DOD_COALESCE' 'EXPIRE_FLAG' 'HADM_ID'
 'ADMITTIME' 'DISCHTIME' 'DEATHTIME' 'DIAGNOSIS' 'HAS_CHARTEVENTS_DATA']
['SUBJECT_ID' 'GENDER' 'DOB' 'DOD_COALESCE' 'EXPIRE_FLAG']
['SUBJECT_ID' 'GENDER' 'DOB' 'DOD_COALESCE' 'EXPIRE_FLAG' 'HADM_ID'
 'ADMITTIME' 'DISCHTIME' 'DEATHTIME' 'DIAGNOSIS' 'HAS_CHARTEVENTS_DATA'
 'age']
89.0
16.0
Just removed 9861 patients who were older than 89 or younger than 16, 36587 patients left in the dataset.


Step 3: Add indicator for number of ICU staye per hospital admissions. Do not count readmission for sepsis analysis

In [7]:
# Add indicator for multiple ICU_stays per HADM
beforeStays = icustays_df["ICUSTAY_ID"].nunique()
counts_icustays_df = icustays_df.groupby(['SUBJECT_ID', 'HADM_ID'])['ICUSTAY_ID'].count().reset_index(drop=False)
counts_icustays_df = counts_icustays_df[counts_icustays_df["ICUSTAY_ID"] == 1]
icustays_df = pd.merge(icustays_df, counts_icustays_df.drop(['ICUSTAY_ID'], axis=1), how='inner', on=['SUBJECT_ID', 'HADM_ID'])
afterStays = icustays_df["ICUSTAY_ID"].nunique()

print("After removing " + str(beforeStays - afterStays) + " ICU stays, there are now " + str(afterStays) + " stays left.")

After removing 2952 ICU stays, there are now 20668 stays left.


Step 4: Filter out certain service types.

In [8]:
services_df['indicator_service'] = services_df['CURR_SERVICE'].apply(lambda x: 
                                                                 0 if x in set(['CSURG','VSURG','TSURG']) 
                                                                 else 1)
beforeServices = len(services_df["HADM_ID"])
services_df = services_df[services_df["indicator_service"] == 1]
afterServices = len(services_df["HADM_ID"])
print(services_df.columns.values)
print("After filtering out " + str(beforeServices - afterServices) + " services, there are now " + str(afterServices) + " left.")

['SUBJECT_ID' 'HADM_ID' 'CURR_SERVICE' 'indicator_service']
After filtering out 10704 services, there are now 62639 left.


 Step 5: Join the dataframes.

In [9]:

# Join Patients, Admissions, ICU information together to build an initial cohort that contains all patients
cohort_df = patients_df.merge(icustays_df, how = "inner", on = ["HADM_ID", "SUBJECT_ID"])
cohort_df = cohort_df.merge(services_df, how = "inner", on = ['HADM_ID','SUBJECT_ID'])
print(cohort_df.columns.values)
cohort_df = cohort_df.drop(["LAST_CAREUNIT"], axis = 1)
cohort_df = cohort_df.drop(["FIRST_WARDID"], axis = 1)
cohort_df = cohort_df.drop(["LAST_WARDID"], axis = 1)
cohort_df = cohort_df.drop(["DBSOURCE"], axis = 1)
cohort_df = cohort_df.drop(["CURR_SERVICE"], axis = 1)
cohort_df = cohort_df.drop(["EXPIRE_FLAG"], axis = 1)

print("There are " + str(cohort_df["SUBJECT_ID"].nunique()) + " patients left after merging.")

['SUBJECT_ID' 'GENDER' 'EXPIRE_FLAG' 'HADM_ID' 'ADMITTIME' 'DISCHTIME'
 'DEATHTIME' 'DIAGNOSIS' 'age' 'ICUSTAY_ID' 'DBSOURCE' 'FIRST_CAREUNIT'
 'LAST_CAREUNIT' 'FIRST_WARDID' 'LAST_WARDID' 'INTIME' 'OUTTIME' 'LOS'
 'indicator_dbsource' 'CURR_SERVICE' 'indicator_service']
There are 13818 patients left after merging.


## Step 6: Suspicion

This step is used the idea in the following sql from The Github:
1. cohort.sql
2. suspicion-of-infection.sql
3. abx-micro-prescription.sql

### Step 6.1: Add indicator for suspection

In [ ]:
# Join tables together to prepare suspection analysis

int_cohort_df = pd.merge(pd.merge(cohort_df, prescr_ab_only_df, how='left', left_on=['HADM_ID'], right_on=['hadm_id'])\
                    .drop(['hadm_id'], axis=1), micro_events_df, how='left', left_on=['HADM_ID'], right_on=['hadm_id'])\
                    .drop(['hadm_id'], axis=1)

In [ ]:
# Remove null values
int_cohort_df = int_cohort_df[(int_cohort_df['antibiotic_time'].notnull()) | (int_cohort_df['charttime'].notnull())]

In [ ]:
# Add indicators for early and late suspection

int_cohort_df['indicator_early_suspection'] = int_cohort_df[['antibiotic_time','charttime']].apply(lambda x: 
                                                                  0 if (x['antibiotic_time']<=x['charttime']) 
                                                                  & (x['antibiotic_time']>x['charttime']-pd.Timedelta(24,unit='h')) 
                                                                  else 1, axis=1)
int_cohort_df['indicator_late_suspection'] = int_cohort_df[['antibiotic_time','charttime']].apply(lambda x: 
                                                                  0 if (x['antibiotic_time']<=x['charttime']+pd.Timedelta(72,unit='h')) 
                                                                  & (x['antibiotic_time']>x['charttime']) 
                                                                  else 1, axis=1)

In [ ]:
print('Count the patients before and after selection.')

print('')

print('Total patients before selection: {:5d}.'.format(cohort_df['SUBJECT_ID'].unique().shape[0]))
print('Total patients after all selection: {:4d}.'\
      .format(int_cohort_df[int_cohort_df.filter(regex=("indicator.*")).all(axis=1)]['SUBJECT_ID'].unique().shape[0]))

print('')

print('Total hospital admission before selection: {:5d}.'.format(cohort_df['HADM_ID'].unique().shape[0]))
print('Total hospital admission after all selection: {:4d}.'\
      .format(int_cohort_df[int_cohort_df.filter(regex=("indicator.*")).all(axis=1)]['HADM_ID'].unique().shape[0]))

In [ ]:
final_cohort_df = int_cohort_df[int_cohort_df.filter(regex=("indicator.*")).all(axis=1)].reset_index(drop=True)\
.drop(['antibiotic_name', 'antibiotic_time', 'antibiotic_endtime', 
       'chartdate', 'charttime', 'spec_type_desc', 'positiveculture'], axis=1)\
.drop_duplicates()

print(str(final_cohort_df["SUBJECT_ID"].nunique()))
print(str(final_cohort_df["HADM_ID"].nunique()))

final_cohort_df.to_csv("cohort_new.csv", index = False)